# Interpolation loop

Run for 2 month period, loop over files pulling 7(?) days material, interpolate and calculate adjustemnts

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import datetime
from sklearn import ensemble

In [52]:
## -----------------------
## Prep grid (mostly for bounds)
grid_gdf = gpd.read_file("./data/coarse_grid_pts/grid_pts_coarse.shp")
grid_gdf = grid_gdf.set_crs(4326, allow_override=True)
grid_gdf = grid_gdf.to_crs(32612)

## Grid min/max (replaced by set limits below)
grid_crds = grid_gdf.get_coordinates()
min_x = grid_crds.x.min()
max_x = grid_crds.x.max()
min_y = grid_crds.y.min()
max_y = grid_crds.y.max()

## Updated to help estimate of basis functions
min_x = 393500
max_x = 453500
min_y = 4472000
max_y = 4532000
print(min_x, max_x, min_y, max_y)

## Standardize coordinates
grid_crds['normalized_east'] = (grid_crds['x']-min_x)/(max_x - min_x)
grid_crds['normalized_north'] = (grid_crds['y']-min_y)/(max_y-min_y)

## -----------------------
## Air monitor data
aq_df = pd.read_csv("./data/loop_test/summer23_ozone_stationary.csv")
aq_df['day_time'] = pd.to_datetime(aq_df['day_time']).dt.tz_localize(None)
## add the timezone:
aq_df['day_time'] = aq_df['day_time'] + pd.Timedelta(hours=7)

## Convert to geopandas
aq_gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(aq_df.longitude, aq_df.latitude, crs="EPSG:4326"), data=aq_df
)
aq_gdf = aq_gdf.to_crs(32612)

## Standardize coordinates
aq_crds = aq_gdf.get_coordinates()
lon = aq_crds.x
lat = aq_crds.y
aq_df['normalized_east'] = (lon-min_x)/(max_x - min_x)
aq_df['normalized_north'] = (lat-min_y)/(max_y-min_y)



393500 453500 4472000 4532000


In [54]:
## -----------------------
## EBus data
ebus = pd.read_csv("./data/loop_test/ebus_2023_06-07.csv", header = [0,1],  
                 na_values = -9999.00)
ebus2_df = pd.DataFrame({
    'time': pd.to_datetime(ebus.iloc[:,1]),
    'lon': ebus.iloc[:,2],
    'lat': ebus.iloc[:,3]
    })
ebus2_df['val'] = ebus['O3'] / 1000

ebus2_df

,time,lon,lat,val
0,2023-07-01 00:00:00,40.70350,-111.97723,0.0573
1,2023-07-01 00:00:00,40.69294,-111.96915,0.0604
2,2023-07-01 00:00:00,40.69298,-111.96718,0.0578
3,2023-07-01 00:01:00,40.70347,-111.97723,0.0525
4,2023-07-01 00:01:00,40.69294,-111.97194,0.0610
...,...,...,...,...
177066,2023-06-30 23:58:00,40.69344,-111.96056,0.0560
177067,2023-06-30 23:59:00,40.69312,-111.96054,0.0546
177068,2023-06-30 23:59:00,40.70405,-111.97715,0.0578
177069,2023-06-30 23:59:00,40.76354,-111.90943,0.0589


In [ ]:
## Convert to geopandas
ebus2_gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(ebus2_df.lon, ebus2_df.lat, crs="EPSG:4326"), data=ebus2_df
)
ebus2_gdf = ebus2_gdf.to_crs(32612)

## Standardize coordinates
ebus2_crds = ebus2_gdf.get_coordinates()
lon = ebus2_crds.x
lat = ebus2_crds.y
ebus2_df['normalized_east'] = (lon-min_x)/(max_x - min_x)
ebus2_df['normalized_north'] = (lat-min_y)/(max_y-min_y)

## plt.plot(grid_crds['normalized_east'], grid_crds['normalized_north'], 'bo', markersize=1)
## plt.plot(ebus2_df['normalized_east'], ebus2_df['normalized_north'], 'ro', markersize=2)
## plt.plot(aq_df['normalized_east'], aq_df['normalized_north'], 'go')
## plt.show()


In [ ]:
## -----------------------
## Basis functions

## Function to make knots in 1D space/time [0, 1]. Pass vector of number of knots in 1D
def make_knots_time(n):
    knots = [np.linspace(0,1,int(i)) for i in n]
    return(knots)

## Function to make Gaussian BFs along one dimension
def get_basis_gaussian_1d(s, num_basis, knots, std_arr):
    N = len(s)
    phi = np.zeros((N, sum(num_basis)))
    K = 0
    for res in range(len(num_basis)):
        std = std_arr[res]
        for i in range(num_basis[res]):
            d = np.square(np.absolute(s-knots[res][i]))
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = np.exp(-0.5 * d[j]/(std**2))
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    return(phi)

## Function to make knots in 2D space [0, 1]. Pass vector of number of knots in 2D (i.e. res of 5, pass 25)
def make_knots_space(n):
    knots = [np.linspace(0,1,int(np.sqrt(i))) for i in n]
    return knots

## Function to make Wendland BFs over two dimensions
def get_basis_wendland_2d(s, num_basis, knots_1d):
    ## Get weights from Wendland kernel
    N = len(s)
    K = 0
    phi = np.zeros((N, sum(num_basis)))
    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    return(phi)


In [ ]:
## -----------------------
## Main loop

all_days = aq_df.date.unique()
all_days = all_days[6:]
for i in all_days:
    print("## -------------- ##")
    print(f"Running for {i}")
    target_date = pd.to_datetime(i)
    print(target_date)
    start_date = target_date - pd.to_timedelta(7, unit='d')
    print(start_date)

    ## ------------------------------------------------------------------------
    ## Extract monitor data
    print("Prepping monitor data")
    print(aq_df['day_time'].dtype)
    aq_df_sub = aq_df[(aq_df['day_time'] >= start_date) & (aq_df['day_time'] <= target_date)]
    print(aq_df_sub.shape)
    day_time = aq_df_sub['day_time'].astype('int64') / 1e9 ## Time in nanoseconds
    min_t = day_time.min()
    max_t = day_time.max()
    aq_df_sub['normalized_time'] = (day_time - min_t) / (max_t-min_t)

    ## Create knots for time basis - 14 day basis function (/2 for 7 day)
    num_basis_t = [10,20,56]
    std_arr_t = [0.3,0.15,0.05]
    knots_1d_t = make_knots_time(num_basis_t)

    ## Create time basis for monitor data
    s = np.array(aq_df_sub['normalized_time']).reshape(len(aq_df_sub),1)
    phi_t1 = get_basis_gaussian_1d(s, num_basis_t, knots_1d_t, std_arr_t)
    print(phi_t1.shape)

    ## Knots for spatial dimension (from STDK example)
    num_basis_s = [7**2,13**2,25**2] ## For 60km grid this is 10/5/2.5 resolution
    knots_1d_s = make_knots_space(num_basis_s)
    
    ## Create time basis for monitor data
    s = np.vstack((aq_df_sub['normalized_east'],aq_df_sub['normalized_north'])).T
    phi_s1 = get_basis_wendland_2d(s, num_basis_s, knots_1d_s)
    print(phi_s1.shape)

    phi_1 = np.hstack((phi_t1,phi_s1))
    idx_zero = np.array([], dtype=int)
    for i in range(phi_1.shape[1]):
        if sum(phi_1[:,i]!=0)==0:
            idx_zero = np.append(idx_zero,int(i))
    
    phi_1_reduce = np.delete(phi_1,idx_zero,1)
    print(phi_1.shape)
    print(phi_1_reduce.shape)

    ## ------------------------------------------------------------------------
    ## EBus data
    print("Prepping EBus data")
    start_date_pred = target_date - pd.to_timedelta(1, unit='d')
    print(ebus2_df['time'].dtype)
    print("target_date")
    print(target_date)
    print(target_date.__class__)
    print("start_date_pred")
    print(start_date_pred)
    print(start_date_pred.__class__)
    ebus2_df_sub = ebus2_df[(ebus2_df['time'] > start_date_pred) & (ebus2_df['time'] <= target_date)]
    print(ebus2_df_sub.shape)
    print("## ------------- ##")
    print(ebus2_df['time'] > start_date_pred)
    print("## ------------- ##")
    print(ebus2_df['time'] <= target_date)
    print("## ------------- ##")

    day_time = ebus2_df_sub['time'].astype('int64') / 1e9 ## Time in nanoseconds
    print("time")
    print(ebus2_df['time'])
    print("day_time")
    print(day_time)
    ebus2_df_sub['normalized_time'] = (day_time - min_t) / (max_t-min_t)

    s = np.array(ebus2_df_sub['normalized_time']).reshape(len(ebus2_df_sub),1)
    phi_t2 = get_basis_gaussian_1d(s, num_basis_t, knots_1d_t, std_arr_t)
    print(phi_t2.shape)

    s = np.vstack((ebus2_df_sub['normalized_east'],ebus2_df_sub['normalized_north'])).T
    phi_s2 = get_basis_wendland_2d(s, num_basis_s, knots_1d_s)
    print(phi_s2.shape)

    phi_2 = np.hstack((phi_t2,phi_s2))
    phi_2_reduce = np.delete(phi_2,idx_zero,1)
    print(phi_2.shape)
    print(phi_2_reduce.shape)
    
    ## ------------------------------------------------------------------------
    ## Random forest
    print("Running RF model")
    ## Create arrays for training RF
    X_train = phi_1_reduce
    y_train = aq_df_sub['sample.measurement'].values

    ## Create and train
    #aq_rf = ensemble.RandomForestRegressor()
    #aq_rf.fit(X_train, y_train)

    break

In [ ]:
i = "2023-06-09"
print("## -------------- ##")
print(f"Running for {i}")
target_date = pd.to_datetime(i)
print(target_date)
start_date = target_date - pd.to_timedelta(7, unit='d')
print(start_date)

In [ ]:
## ------------------------------------------------------------------------
## Extract monitor data
print("Prepping monitor data")
print(aq_df['day_time'].dtype)
aq_df_sub = aq_df[(aq_df['day_time'] >= start_date) & (aq_df['day_time'] <= target_date)]
print(aq_df_sub.shape)
day_time = aq_df_sub['day_time'].astype('int64') / 1e9 ## Time in nanoseconds
min_t = day_time.min()
max_t = day_time.max()
aq_df_sub['normalized_time'] = (day_time - min_t) / (max_t-min_t)

## Create knots for time basis - 14 day basis function (/2 for 7 day)
num_basis_t = [10,20,56]
std_arr_t = [0.3,0.15,0.05]
knots_1d_t = make_knots_time(num_basis_t)

## Create time basis for monitor data
s = np.array(aq_df_sub['normalized_time']).reshape(len(aq_df_sub),1)
phi_t1 = get_basis_gaussian_1d(s, num_basis_t, knots_1d_t, std_arr_t)
print(phi_t1.shape)

## Knots for spatial dimension (from STDK example)
num_basis_s = [7**2,13**2,25**2] ## For 60km grid this is 10/5/2.5 resolution
knots_1d_s = make_knots_space(num_basis_s)

## Create time basis for monitor data
s = np.vstack((aq_df_sub['normalized_east'],aq_df_sub['normalized_north'])).T
phi_s1 = get_basis_wendland_2d(s, num_basis_s, knots_1d_s)
print(phi_s1.shape)

phi_1 = np.hstack((phi_t1,phi_s1))
idx_zero = np.array([], dtype=int)
for i in range(phi_1.shape[1]):
    if sum(phi_1[:,i]!=0)==0:
        idx_zero = np.append(idx_zero,int(i))

phi_1_reduce = np.delete(phi_1,idx_zero,1)
print(phi_1.shape)
print(phi_1_reduce.shape)



In [ ]:
## ------------------------------------------------------------------------
## EBus data
print("Prepping EBus data")
start_date_pred = target_date - pd.to_timedelta(1, unit='d')
print(ebus2_df['time'].dtype)
print("target_date")
print(target_date)
print(target_date.__class__)
print("start_date_pred")
print(start_date_pred)
print(start_date_pred.__class__)
ebus2_df_sub = ebus2_df[(ebus2_df['time'] > start_date_pred) & (ebus2_df['time'] <= target_date)]
print(ebus2_df_sub.shape)
print("## ------------- ##")
print(ebus2_df['time'] > start_date_pred)
print("## ------------- ##")
print(ebus2_df['time'] <= target_date)
print("## ------------- ##")



In [ ]:
print(ebus2_df['time'].unique())

In [ ]:
print(ebus2_df['time'][0])
print(start_date_pred)
ebus2_df['time'][0] > start_date_pred

In [ ]:
print(ebus2_df['time'][0])
td2 = pd.to_datetime("2023-07-08")
print(td2)
ebus2_df['time'][0] < td2

In [ ]:
x = ebus2_df['time'] > start_date_pred
x.sum()

In [ ]:
target_date

In [ ]:
x = ebus2_df['time'] <= target_date
x.sum()

In [ ]:
ebus2_df['time'].min()

In [ ]:
day_time = ebus2_df_sub['time'].astype('int64') / 1e9 ## Time in nanoseconds
print("time")
print(ebus2_df['time'])
print("day_time")
print(day_time)
ebus2_df_sub['normalized_time'] = (day_time - min_t) / (max_t-min_t)



In [ ]:
s = np.array(ebus2_df_sub['normalized_time']).reshape(len(ebus2_df_sub),1)
phi_t2 = get_basis_gaussian_1d(s, num_basis_t, knots_1d_t, std_arr_t)
print(phi_t2.shape)

s = np.vstack((ebus2_df_sub['normalized_east'],ebus2_df_sub['normalized_north'])).T
phi_s2 = get_basis_wendland_2d(s, num_basis_s, knots_1d_s)
print(phi_s2.shape)

phi_2 = np.hstack((phi_t2,phi_s2))
phi_2_reduce = np.delete(phi_2,idx_zero,1)
print(phi_2.shape)
print(phi_2_reduce.shape)

## ------------------------------------------------------------------------
## Random forest
print("Running RF model")
## Create arrays for training RF
X_train = phi_1_reduce
y_train = aq_df_sub['sample.measurement'].values

## Create and train
#aq_rf = ensemble.RandomForestRegressor()
#aq_rf.fit(X_train, y_train)